In [15]:
import pandas as pd
import nltk
import re
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
stemmer=PorterStemmer()
from kerastuner.tuners import RandomSearch

In [24]:
df=pd.read_csv(r"C:\Users\ASUS\Desktop\sms.txt",sep='\t',names=["Result","Message"])

In [25]:
for i in range(len(df)):
    words=nltk.word_tokenize(df['Message'][i])
    words = re.sub('[^a-zA-Z]', ' ', df['Message'][i])
    words=words.lower()
    words=words.split(" ")
    words=[stemmer.stem(word) for word in words if word not in stopwords.words("english")]
    df['Message'][i]="  ".join(words)
df


,Result,Message
0,ham,go jurong point crazi avail bugi n...
1,ham,ok lar joke wif u oni
2,spam,free entri wkli comp win fa cup fin...
3,ham,u dun say earli hor u c alreadi ...
4,ham,nah think goe usf live around though
...,...,...
5567,spam,nd time tri contact u u ...
5568,ham,b go esplanad fr home
5569,ham,piti mood suggest
5570,ham,guy bitch act like interest buy someth ...


In [26]:
X=df.iloc[:,-1]

In [27]:
Y=pd.get_dummies(df['Result'],drop_first=True)

In [28]:
from sklearn.feature_extraction.text import CountVectorizer
cv=CountVectorizer(max_features=3000)

In [29]:
X=cv.fit_transform(X).toarray()
X

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [30]:
Y.shape

(5572, 1)

In [31]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,Y,test_size=0.2,random_state=0)

In [32]:
import tensorflow as tf
from tensorflow import keras

In [82]:
def build_model(hp):
    model = tf.keras.Sequential()
    for i in range(hp.Int('num_layers', 2, 8)):
        model.add(tf.keras.layers.Dense(units=hp.Int('units_' + str(i),
                                            min_value=264,
                                            max_value=1024,
                                            step=32),
                               activation='relu'))
    model.add(tf.keras.layers.Dense(1, activation='sigmoid'))
    model.compile(
        optimizer=keras.optimizers.Adamax(
            hp.Choice('learning_rate', [1e-2, 1e-3, 1e-4])),
        loss='binary_crossentropy',
        metrics=['accuracy'])
    return model

In [83]:
tuner = RandomSearch(
    build_model,
    objective='accuracy',
    max_trials=5,
    executions_per_trial=3,
    directory='Desktop',
    project_name='new_classifier')

In [84]:
tuner.search_space_summary()


Search space summary
Default search space size: 4
num_layers (Int)
{'default': None, 'conditions': [], 'min_value': 2, 'max_value': 8, 'step': 1, 'sampling': None}
units_0 (Int)
{'default': None, 'conditions': [], 'min_value': 264, 'max_value': 1024, 'step': 32, 'sampling': None}
units_1 (Int)
{'default': None, 'conditions': [], 'min_value': 264, 'max_value': 1024, 'step': 32, 'sampling': None}
learning_rate (Choice)
{'default': 0.01, 'conditions': [], 'values': [0.01, 0.001, 0.0001], 'ordered': True}


In [85]:
tuner.search(X_train, y_train,validation_data=(X_test,y_test),verbose=2,
             epochs=10)

Trial 5 Complete [00h 06m 58s]
accuracy: 1.0

Best accuracy So Far: 1.0
Total elapsed time: 00h 22m 55s
INFO:tensorflow:Oracle triggered exit


In [86]:
model=tf.keras.Sequential()
model.add(tf.keras.layers.Dense(968,activation='relu',input_shape=(3000,)))
model.add(tf.keras.layers.Dense(872,activation='relu'))
model.add(tf.keras.layers.Dense(1,activation='sigmoid'))

In [87]:
model.compile(optimizer='Adamax',loss='binary_crossentropy',metrics=['accuracy'])

In [88]:
model.fit(X_train,y_train, epochs=10)

Epoch 1/10
140/140 [==============================] - 10s 73ms/step - loss: 0.1850 - accuracy: 0.9367
Epoch 2/10
140/140 [==============================] - 11s 77ms/step - loss: 0.0348 - accuracy: 0.9895
Epoch 3/10
140/140 [==============================] - 12s 83ms/step - loss: 0.0174 - accuracy: 0.99483s - loss: 0.017 - ETA: 2s - l - ETA
Epoch 4/10
140/140 [==============================] - 11s 81ms/step - loss: 0.0102 - accuracy: 0.9975
Epoch 5/10
140/140 [==============================] - 11s 77ms/step - loss: 0.0067 - accuracy: 0.9984
Epoch 6/10
140/140 [==============================] - 10s 70ms/step - loss: 0.0045 - accuracy: 0.9989
Epoch 7/10
140/140 [==============================] - 11s 80ms/step - loss: 0.0030 - accuracy: 0.9993
Epoch 8/10
140/140 [==============================] - 12s 85ms/step - loss: 0.0021 - accuracy: 0.9996
Epoch 9/10
140/140 [==============================] - 13s 91ms/step - loss: 0.0015 - accuracy: 0.9996
Epoch 10/10
140/140 [=========================

In [89]:
model.evaluate(X_test, y_test, verbose=2)

35/35 - 1s - loss: 0.0693 - accuracy: 0.9901


[0.06929577887058258, 0.9901345372200012]

In [90]:
predictions = model.predict(X_test)

In [91]:
b=list(predictions)
for i in range(len(b)):
    if b[i]>0.5:
        b[i]=1
    else:
        b[i]=0


In [92]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test,b)

array([[954,   1],
       [ 10, 150]], dtype=int64)

In [93]:
from sklearn.metrics import classification_report
print(classification_report(y_test, b))

              precision    recall  f1-score   support

           0       0.99      1.00      0.99       955
           1       0.99      0.94      0.96       160

    accuracy                           0.99      1115
   macro avg       0.99      0.97      0.98      1115
weighted avg       0.99      0.99      0.99      1115

